In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
import scqubits as scq

ModuleNotFoundError: No module named 'scqubits'

In [3]:
import sympy as sp
import numpy as np

x = sp.Symbol('x')
p = sp.Symbol('p')
m = sp.Symbol('m')
omega = sp.Symbol('omega')

s = (m*omega*x+1j*p)/sp.sqrt(2*m*omega)
sdag = (m*omega*x-1j*p)/sp.sqrt(2*m*omega)

sp.simplify(sdag*sdag*s + sdag*s*s)

0.5*sqrt(2)*x*(m**2*omega**2*x**2 + p**2)/sqrt(m*omega)

In [4]:
sp.expand(sp.simplify(sdag*sdag*s*s))

m**2*omega**2*x**4/4 + 0.5*p**2*x**2 + 0.25*p**4/(m**2*omega**2)

In [5]:
a = destroy(5)

In [6]:
wq = 3.6e9*2*np.pi
alpha = 223e6*2*np.pi

In [7]:
H = wq*a.dag()*a - alpha/2*a.dag()*a.dag()*a*a

In [8]:
H.eigenenergies()/(2*np.pi)
np.diff(H.eigenenergies()/(2*np.pi)) 

array([3.600e+09, 3.377e+09, 3.154e+09, 2.931e+09])

In [11]:
gdelta2 = 0.0000917
energies = 5.729e9 - gdelta2*H.eigenenergies()/(2*np.pi)
np.diff(energies)/1e6

array([-0.33012  , -0.3096709, -0.2892218, -0.2687727])

In [12]:
import numpy as np
import scqubits as scq

def tmon_model(EJ, EC):

    tmon = scq.Transmon(
        EJ=EJ,
        EC=EC,
        ng=0.0,
        ncut=31
    )
    
    tmon.eigenvals(evals_count=12)

    freq = np.diff(tmon.eigenvals(evals_count=12))[0]
    alpha = np.diff(np.diff(tmon.eigenvals(evals_count=12)))[0]

    return freq, alpha

def cost(x, *args):
    
    EJ = x[0]
    EC = x[1]
    
    freq_exp = args[0][0]
    alpha_exp = args[0][1]

    
    freq_model, alpha_model = tmon_model(EJ, EC)
    
    exp = np.array([freq_exp, alpha_exp])
    model = np.array([freq_model, alpha_model])
    
    error = np.sum( ((model-exp) / exp)**2 )
    
    return error

import scipy.optimize

def get_transmon_hamiltonian(freq, alpha):

    x0 = np.array([9.3, 0.195])

    args = [freq, alpha]

    result = scipy.optimize.minimize(cost, x0, tol=1e-8,bounds=[(0,100),(0,100)],args=args)

    result['x']

    tmon = scq.Transmon(
            EJ=9.44645349,
            EC=0.19360087,
            ng=0.0,
            ncut=31)
    
    return tmon.hamiltonian()

get_transmon_hamiltonian(3.62e9, 223e6)

array([[744.20174428,  -4.72322674,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [ -4.72322674, 696.963132  ,  -4.72322674, ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,  -4.72322674, 651.27332668, ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [  0.        ,   0.        ,   0.        , ..., 651.27332668,
         -4.72322674,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,  -4.72322674,
        696.963132  ,  -4.72322674],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
         -4.72322674, 744.20174428]])